We want to count [A239760](https://oeis.org/A239760) the number of pairs of functions $(f,g)$ on $[n]$ such that $ffg=f$.  Note that the depth of a tree in the functional graph of $f$ is at most one, since if the maximum depth point $x$ is greater than 1, then $g(x)$ must have depth one greater, contradicting the assumption that $x$ was the maximum depth point.

So $f$ consists of a bunch of fuzzy cycles, so let us look at the number of functions $g$ such that $ffg=f$ where the functional graph of $f$ is a cycle with $k$ vertices.  Let us label the points on the cycle $v_1,v_2,\dots,v_k$ in cyclic order. If for $v_i$ on the cycle $f^{-1}(v_i)$ has cardinality $x_i$, where $x_i\ge 1$ since $v_i$ has at least its predecessor on the cycle, then the number of $g$ that satisfy $ffg=f$ is $x_1^{x_2}x_2^{x_3}\cdots x_k^{x_1}$ since each predecessor of $v_i$ can map to any predecessor of $v_{i-1}$ (where the subtraction is modulo the cycle.

Given $n$ elements and $k$ points on the cycle, there are $\binom{n}{k}$ ways of choosing the elements on the cycle, and $(k-1)!$ ways of ordering the cycle and for any composition of $n-k$ into $k$ parts $y_1,y_2,\dots,y_k$ there are $\binom{n-k}{y_1,y_2,\dots,y_k}$ ways of doing this and ${(y_1+1)}^{y_2+1}{(y_2+1)}^{y_3+1}\cdots{(y_k+1)}^{y_1+1}$ ways of assigning $g$.  So altogether
$$\sum_{k=1}^n \binom{n}{k}(k-1)!\sum_{y_1+y_2+\cdots+y_k=n-k}\binom{n-k}{y_1,y_2,\dots,y_k}\prod_{i=1}^k{(y_i+1)}^{y_{i+1}+1}$$
where $y_{k+1}=y_1$.

We can then use eqn (5.7) from EC2 to get the count with any number of connected components.

This is probably best computed using dynamic programming, but let us compute it directly first.

In [53]:
def arrange_single_component(n,k):
    return sum(multinomial(y)*prod((y[i]+1)^(y[(i+1) % k]+1) for i in srange(k)) for y in IntegerVectors(n-k,k))

@CachedFunction
def count_single_component(n):
    return sum(binomial(n,k) * factorial(k-1) * arrange_single_component(n,k) for k in srange(1, n+1))

@CachedFunction
def a(n):
    if n == 0:
        return 1
    else:
        return sum(a(k) * count_single_component(n - k) * binomial(n-1, k) for k in srange(n))


print [count_single_component(n) for n in srange(1,10)]
print [a(n) for n in srange(19)]
    
    

[1, 9, 95, 1306, 21429, 417216, 9384283, 239973840, 6887864169]
[1, 1, 10, 123, 1984, 38765, 899856, 24135055, 734381824, 24985937241, 939674886400, 38697245287811, 1731506682516480, 83631998873272837, 4336190248653942784, 240186479159794548375, 14153996066789439471616, 884109266328681103701425, 58346867827408911965945856]


For a dynamic programming approach, we want to speed the computation of the sum over the exponentially many compositions of $n-k$ into $k$ parts.  Let us call the inner sum over the compositions into $k$ parts, $S$, so that
$$S=\sum_{y_1+y_2+\cdots+y_k=n-k}\binom{n-k}{y_1,y_2,\dots,y_k}\prod_{i=1}^k{(y_i+1)}^{y_{i+1}+1}$$

and rewrite it as
$$S=\sum_{y_1+y_k\le n-k}{(y_k+1)}^{y_1+1}(n-k)!\frac{1}{y_k!}\sum_{y_1+y_2+\cdots+y_k=n-k}\prod_{i=1}^{k-1}\frac{{(y_i+1)}^{y_{i+1}+1}}{y_i!}.$$

Now if we define a function $b$ to represent the inner sum
$$b(j,y_1,y_j,s)=\frac{1}{y_j!}\sum_{y_1+y_2+\cdots+y_j=s}\prod_{i=1}^{j-1}\frac{{(y_i+1)}^{y_{i+1}+1}}{y_i!}$$

then our sum becomes
$$S=\sum_{y_1+y_k\le n-k}(n-k)!{(y_k+1)}^{y_1+1} b(k,y_1,y_k,n-k).$$


If $k=1$ or $k=2$ then computing the number of pairs where $f$ has a single component and cycle length $k$ is easy, so we only need to compute $b$ for $j\ge 3$.
If $j=3$ then for $y_1+y_3\le s$ we have:
$$b(3,y_1,y_3,s) = \frac{{(y_1+1)}^{s-y_1-y_3+1}{(s-y_1-y_3+1)}^{y_3+1}}{y_1!(s-y_1-y_3)!y_3!}$$
and for $j>3$ we have recursively:
$$b(j,y_1,y_j,s) = \frac{1}{y_j!}\sum_{y_{j-1}=0}^{s-y_1-y_j} {(y_{j-1}+1)}^{y_j+1}b(j-1,y_1,y_{j-1},s-y_j) .$$


It is clear that we cannot memoize everything, as four variables are still way too many, the amount of space grows more than quarticly in $n$ but this should be enough to get up to a hundred or so.

In [50]:
@CachedFunction
def r(j, y1, yj, s):
    if j == 3:
        y = s - y1 - yj
        return Integer((y+1))^(yj+1) * Integer((y1+1))^(y+1) / (factorial(y1) * factorial(y) * factorial(yj))
    else:
        return sum((y+1)^(yj+1) * r(j-1, y1, y, s-yj) for y in IntegerRange(0, s-y1-yj+1)) / factorial(yj)
        

def memoize_single_component(n, k):
    if k == 1:
        return n^n
    elif k == 2:
        return sum(binomial(n-2,j)*(j+1)^(n-1-j)*(n-1-j)^(j+1) for j in srange(n-1))
    else:
        return sum((yk+1)^(y1+1) * factorial(n-k) * r(k, y1, yk, n-k) 
                   for y1 in IntegerRange(n-k+1) for yk in srange(n-k-y1+1))
    
print all(memoize_single_component(n, k) == arrange_single_component(n, k) for n in srange(1,8) for k in srange(1,5))    

True


Ok, things seem to be working so far.

Lets try everything together and see how far we get

In [51]:
@CachedFunction
def count_single_component2(n):
    return sum(binomial(n,k) * factorial(k-1) * memoize_single_component(n,k) for k in srange(1, n+1))

@CachedFunction
def a2(n):
    if n == 0:
        return 1
    else:
        return sum(a2(k) * count_single_component2(n - k) * binomial(n-1, k) for k in srange(n))

print [count_cycles2(n) for n in srange(1,10)]
print [a2(n) for n in srange(70)]
print len(r.cache)

[1, 9, 95, 1306, 21429, 417216, 9384283, 239973840, 6887864169]
[1, 1, 10, 123, 1984, 38765, 899856, 24135055, 734381824, 24985937241, 939674886400, 38697245287811, 1731506682516480, 83631998873272837, 4336190248653942784, 240186479159794548375, 14153996066789439471616, 884109266328681103701425, 58346867827408911965945856, 4056531481769213400939687691, 296338303357039938312110080000, 22693228248817732322565519909021, 1817836341939316825845804150292480, 152026610084316452666986290641059103, 13250078455990342411114160637905731584, 1201549176673010442561145424929368015625, 113196851298904027050920119818653349707776, 11063465327939283799129215922954513267678995, 1120340247937006887722810070594821578150641664, 117405235657577269336294783294350025009913849141, 12717905021036444122626131951767711792496640000000, 1422583032679933489159556204795430117029981916750631, 164151734412545145483200520525783067159598703428239360, 19521666371081628588220724121574942335185539214589321057, 239063447261781

In [52]:
#Lets make a b-file

for i in srange(101):
    if i == 0:
        print 0, 1
    else:
        print i, a2(i)

0 1
1 1
2 10
3 123
4 1984
5 38765
6 899856
7 24135055
8 734381824
9 24985937241
10 939674886400
11 38697245287811
12 1731506682516480
13 83631998873272837
14 4336190248653942784
15 240186479159794548375
16 14153996066789439471616
17 884109266328681103701425
18 58346867827408911965945856
19 4056531481769213400939687691
20 296338303357039938312110080000
21 22693228248817732322565519909021
22 1817836341939316825845804150292480
23 152026610084316452666986290641059103
24 13250078455990342411114160637905731584
25 1201549176673010442561145424929368015625
26 113196851298904027050920119818653349707776
27 11063465327939283799129215922954513267678995
28 1120340247937006887722810070594821578150641664
29 117405235657577269336294783294350025009913849141
30 12717905021036444122626131951767711792496640000000
31 1422583032679933489159556204795430117029981916750631
32 164151734412545145483200520525783067159598703428239360
33 19521666371081628588220724121574942335185539214589321057
34 2390634472617815732

This has not gotten us much mathematically, but enough so that we can conjecture that $a(n)$ is divisible by $n$. We could extend this to compute the number of pairs such that fffg=f and so forth, but that would involve keeping track of even more state.